# Implementation of data compression of random bit strings

Given block length $n$, we select the $M$ most probable bit sequences of length $n$, and encoding them by indices from 1 to $M$, resulting in a compression rate is $M/2^n$.


### Combinatorial number system

The encoding is based on a combinatorial number system.

For each positive integer $k$, there is a bijection between the set of nonnegative integers $\{0,1,2,3,\ldots\}$ and the set of increasing integers
$$
0\leq c_1 < c_2 < c_3 < \cdots < c_k,
$$
The correspondence is given by
$$
	m = \binom{c_1}{1} + \binom{c_2}{2} + \cdots + \binom{c_k}{k}.
$$

In [1]:
from math import comb
from numpy import random, log2, ceil, floor

# Combinatorial number system

# convert a list of bits to integer
def encode_bits_to_int(x : list [int]):   # input x is a list of 0 and 1
  """  transform a list of 0 and 1 into an integer  
  """
  loc = [i for i , m in enumerate(x) if m==1]  # locations of 1
  return sum([comb(m,i+1) for i,m in enumerate(loc)])

# convert an integer to a list of bits
def decode_int_to_bits(a : int, n : int, k : int):
  """  the reverse transform for combinatorial number system
       return a list of length n containing k ones
  """
  d=[]    # initialize to empty list
  for i in range(n-1,-1,-1):  
    binom_coeff = comb(i,k)  # compute binomial coefficient i choose k
    if binom_coeff <= a:   # if this binomial coeff is less than a 
      d.append(i)          # there is a 1 located at i
      a -= binom_coeff     # update variable a
      k -= 1               # update variable k
  return  [1 if j in d else 0 for j in range(n)]

Using combinatorial number system, we can encode the $\binom{n}{k}$ different ways of placing $k$ ones in $n$ positions. The transformation is done by `decode_int_to_bits(i,n,k)`, for $i$ from 0 to $\binom{n}{k}-1$.

In [2]:
# Example of encoding 2 ones in 4 locations
n = 5
k = 2
print("Binary string    | integral representation")
print("-----------------+-----------------")
for i in range(comb(n,k)):
  x = decode_int_to_bits(i,n,k)
  loc = [i for i in range(n) if x[i]==1]
  print(f"{x}  |    {i}")

Binary string    | integral representation
-----------------+-----------------
[1, 1, 0, 0, 0]  |    0
[1, 0, 1, 0, 0]  |    1
[0, 1, 1, 0, 0]  |    2
[1, 0, 0, 1, 0]  |    3
[0, 1, 0, 1, 0]  |    4
[0, 0, 1, 1, 0]  |    5
[1, 0, 0, 0, 1]  |    6
[0, 1, 0, 0, 1]  |    7
[0, 0, 1, 0, 1]  |    8
[0, 0, 0, 1, 1]  |    9


In [3]:
# Example of encoding 5 ones in 7 locations
n = 7
k = 5
print("Binary string          | integral representation")
print("-----------------------+-----------------")
for i in range(comb(n,k)):
  x = decode_int_to_bits(i,n,k)
  loc = [i for i in range(n) if x[i]==1]
  print(f"{x}  |    {i}")

Binary string          | integral representation
-----------------------+-----------------
[1, 1, 1, 1, 1, 0, 0]  |    0
[1, 1, 1, 1, 0, 1, 0]  |    1
[1, 1, 1, 0, 1, 1, 0]  |    2
[1, 1, 0, 1, 1, 1, 0]  |    3
[1, 0, 1, 1, 1, 1, 0]  |    4
[0, 1, 1, 1, 1, 1, 0]  |    5
[1, 1, 1, 1, 0, 0, 1]  |    6
[1, 1, 1, 0, 1, 0, 1]  |    7
[1, 1, 0, 1, 1, 0, 1]  |    8
[1, 0, 1, 1, 1, 0, 1]  |    9
[0, 1, 1, 1, 1, 0, 1]  |    10
[1, 1, 1, 0, 0, 1, 1]  |    11
[1, 1, 0, 1, 0, 1, 1]  |    12
[1, 0, 1, 1, 0, 1, 1]  |    13
[0, 1, 1, 1, 0, 1, 1]  |    14
[1, 1, 0, 0, 1, 1, 1]  |    15
[1, 0, 1, 0, 1, 1, 1]  |    16
[0, 1, 1, 0, 1, 1, 1]  |    17
[1, 0, 0, 1, 1, 1, 1]  |    18
[0, 1, 0, 1, 1, 1, 1]  |    19
[0, 0, 1, 1, 1, 1, 1]  |    20


In the application to data compression, we encode all bit patterns with numbers of bits less than or equal to `max_no_of_bits`.

In [4]:
# an implementation of data compression

# encode bit string to an integer
def encode (bits : list[int], n : int, max_no_of_bits: int) ->int:
  """ bits : a list of zero and one
      n    : length of bits
      max_no_of_bits : maximum number of bits 
  """
  k = sum(bits)  # number of 1 in k
  if k > max_no_of_bits:  # if the number of 1 in c exceeds the limit
    W = sum([comb(n,i) for i in range(max_no_of_bits+1)]) # the error index
  else:
    W = encode_bits_to_int(bits)
    W += sum([comb(n,i) for i in range(k)])
  return W


# Recover the original bits
def decode(n : int , W : int ) -> list[int]: 
  """
   n is the block length and W is the integral representation
   return a list of length n
  """
  k = 0  # determine the number of 1
  while W >= comb(n,k):
    W -= comb(n,k)
    k += 1
  return decode_int_to_bits(W,n,k)

For example, we can encode a bit pattern `[0,1,0,1,0,0,0,1,1,0]` into an integer `a` by command `a = encode(c, n, mb)`, and recover the bit pattern by `decode(n,a)`

In [5]:
c = [0,1,0,1,0,0,0,1,1,0]
mb = 5
n = len(c)
a = encode(c, n, mb)
b = decode(n,a)
print(a)
print(b)

285
[0, 1, 0, 1, 0, 0, 0, 1, 1, 0]


#### Simulation of data compression of random bit strings

In [6]:
# binary entropy function 
def h2(p):
  return -p*log2(p) - (1-p)*log2(1-p)

# simulate
def simulate(p:float, n:int ,delta:float , no_of_runs:int = 1000):
    """
    p : probability of 1
    n : block length
    delta : redundancy
    no_of_runs : repeat the experiment `no_of_runs` times

    return the probability of decoding error, and the number of codewords
    """
    max_no_of_one = int(floor(p*((1+delta))*n))
    M = sum([comb(n,i) for i in range(max_no_of_one+1)]) # no. of codewords    
    error_count = 0
    for _ in range(no_of_runs):   # repeat no_of_runs times
        raw_data_bits = [1 if random.uniform(0,1) < p else 0 for _ in range(n)] # random bits
        W = encode(raw_data_bits, n, max_no_of_one)  # encoded integer
        decoded_bits = decode(n,W)  # recover the original bits from W
        if raw_data_bits != decoded_bits:  # check if there is any error
            error_count += 1   # add one if there is a decoding error
    return error_count, M

In [7]:
n=1000   # block length
p=0.1    # probability of 1
N = 2000 
error_count, M = simulate(p, n, 0.15, N)  # simulate by encoding and decoding N blocks

B = log2(float('0.'+str(M)))+ len(str(M))*log2(10)
print(f"Entropy h2({p}) = {h2(p):.4}")
print(f"Compression rate = {B/n}")
print(f"Encode {n} bits to {ceil(log2(float(M)))} bits.")
print(f"Prob of encoding error = {error_count/N}")

Entropy h2(0.1) = 0.469
Compression rate = 0.5074067079311942
Encode 1000 bits to 508.0 bits.
Prob of encoding error = 0.064
